# Importar Librerías

In [2]:
import datetime
import pickle
import polars as pl
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar Datos

In [68]:
cols_df1 = [
    'CLIENTEAFILIADOBENEFICIARIO',
    'AUTORIZACION',
    'FECHASERVICIO',
    'TIPOATENCION',
    'PRESTADOR',
    'TipoPrestador',
    'ESPECIALIDAD',
    'PRESTACION',
    'TIPOPRESTACION',
    'SERVICIOSALUD',
    'MONTOUNITARIO',
    'CANTIDAD',
    'MONTORECLAMADO',
    'MONTOAJUSTADO',
    'MONTODIFERENCIAAFILIADO',
    'MONTOCOPAGO',
    'AUTORIZACIONRELACIONADA',
    'MONTOASEGURADORA',
    'TipoInstitucionPrestador'
    ]

cols_df2 = [
    'CLIENTEAFILIADOBENEFICIARIO',
    'AUTORIZACION',
    'FECHASERVICIO',
    'TIPOATENCION',
    'PRESTADOR',
    'TipoPrestador',
    'ESPECIALIDAD',
    'PRESTACION',
    'TIPOPRESTACION',
    'SERVICIOSALUD',
    'MONTOAJUSTADO',
    'MONTOCOPAGO',
    'AUTORIZACIONRELACIONADA',
    'MONTOASEGURADORA',
    'MONTOEXCEPCION',
    'MONTOCONCESION',
    'MONTOAUTORIZADO'
    ]

n = 100000

In [69]:
# Data Autorizaciones

df1 = pl.read_parquet(
    '../Datos/dataAutorizaciones20221208.parquet',
    columns = cols_df1
)

df1 = df1.head(n)
df1 = df1.to_pandas()

print(type(df1))
print(df1.shape)

<class 'pandas.core.frame.DataFrame'>
(100000, 19)


In [70]:
# Data Distribuciones Autorizaciones

df2 = pl.read_parquet(
    '../Datos/dataDistribucionesAutorizaciones20221208.parquet',
    columns = cols_df2
)

df2 = df2.head(n)
df2 = df2.to_pandas()

print(type(df2))
print(df2.shape)


<class 'pandas.core.frame.DataFrame'>
(100000, 17)


In [71]:
# CUPS frecuentes en emergencias
cups_df = pd.read_excel(
    '../Datos/CUPS frecuencias y otros (002).xlsx',engine='openpyxl'
)

# CUPS cantidades esperadas
valores_referencia_cups_df = pd.read_excel(
    '../Datos/CUPS frecuencias y otros (002).xlsx', 
    sheet_name=1,engine='openpyxl'
)

# Indicador de dias de habitación

# Read File
salas_df = pd.read_excel(
    '../Datos/salas.xlsx', engine='openpyxl'
)

salas_list = salas_df['DESCRIPCION'].tolist()

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# Unir Datasets

In [72]:
def consolidate_dataframes(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    # Encuentra intersección de autorizaciones
    autorizaciones = set(df1.AUTORIZACION.unique()).intersection(df2.AUTORIZACION.unique())

    # Filtrar y seleccionar columnas relevantes
    selected_columns_df1 = [
    'CLIENTEAFILIADOBENEFICIARIO',
    'AUTORIZACION',
    'FECHASERVICIO',
    'TIPOATENCION',
    'PRESTADOR',
    'TipoPrestador',
    'ESPECIALIDAD',
    'PRESTACION',
    'TIPOPRESTACION',
    'SERVICIOSALUD',
    'MONTOUNITARIO',
    'CANTIDAD',
    'MONTORECLAMADO',
    'MONTOAJUSTADO',
    'MONTODIFERENCIAAFILIADO',
    'MONTOCOPAGO',
    'AUTORIZACIONRELACIONADA',
    'MONTOASEGURADORA',
    'TipoInstitucionPrestador'
    ]
    
    selected_columns_df2 = [
    'CLIENTEAFILIADOBENEFICIARIO',
    'AUTORIZACION',
    'FECHASERVICIO',
    'TIPOATENCION',
    'PRESTADOR',
    'TipoPrestador',
    'ESPECIALIDAD',
    'PRESTACION',
    'TIPOPRESTACION',
    'SERVICIOSALUD',
    'MONTOAJUSTADO',
    'MONTOCOPAGO',
    'AUTORIZACIONRELACIONADA',
    'MONTOASEGURADORA',
    'MONTOEXCEPCION',
    'MONTOCONCESION',
    'MONTOAUTORIZADO'
    ]
    
    filtered_df1 = df1[df1.AUTORIZACION.isin(autorizaciones)][selected_columns_df1]
    filtered_df2 = df2[df2.AUTORIZACION.isin(autorizaciones)][selected_columns_df2]

    # Agrupar montos en el dataset de distribución
    groupby_columns = [
        'AUTORIZACION',
        'CLIENTEAFILIADOBENEFICIARIO',
        'PRESTACION',
        'PRESTADOR',
        'FECHASERVICIO'
    ]
    
    aggregated_df2 = filtered_df2.groupby(groupby_columns)[['MONTOASEGURADORA', 'MONTOEXCEPCION', 'MONTOCONCESION', 'MONTOAUTORIZADO']].sum().reset_index()

    # Consolidar las dos bases de datos
    consolidated_df = filtered_df1.merge(
        aggregated_df2,
        on=groupby_columns
    )
    
    # Eliminar columnas
    consolidated_df.drop(columns=['MONTOASEGURADORA_x'], inplace=True)

    # Renombrar columnas
    
    consolidated_df.rename(
        columns={
            'MONTOASEGURADORA_y': 'MONTOASEGURADORA',
            'MONTODIFERENCIAAFILIADO': 'MONTOUSUARIO',
            'TipoPrestador': 'TIPOPRESTADOR',
        },
        inplace=True
    )
    
    return consolidated_df

# Uso de la función
df = consolidate_dataframes(df1, df2)



In [8]:

print(df.shape)

(17614, 22)


In [ ]:
del(df1, df2)

# Preprocesamiento

In [73]:
def clean_and_format_data(df):
    df['PRESTACION'] = df['PRESTACION'].str.strip() # Eliminar espacios al final y al comienzo de Prestacion
    df['FECHASERVICIO'] = pd.to_datetime(df['FECHASERVICIO']) # Dar formato a Fechas de Servicio
    # Obtener mes de fecha de servicio
    df['FECHASERVICIO_MONTH'] = df['FECHASERVICIO'].dt.month_name() 
    df['FECHASERVICIO_MONTH_NUMBER'] = df['FECHASERVICIO'].dt.month
    
    
clean_and_format_data(df)    

In [ ]:
df.shape

In [ ]:
df.head(1)

In [ ]:
df.loc[df['AUTORIZACION']==46857903].head(1)

In [74]:
def diferencia_montos(df):
    df['DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO'] = df['MONTORECLAMADO'] - df['MONTOAUTORIZADO'] ### Calcular Diferencia de montos reclamado y autorizado
    df['DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO_USUARIO'] = df['MONTORECLAMADO'] - (df['MONTOAUTORIZADO'] + df['MONTOUSUARIO'])### Calcular Diferencia de montos reclamado y autorizado y usuario
    
    ### Calcular Valores Absolutos
    df['MONTOAJUSTADO_ABS'] = abs(df['MONTOAJUSTADO'])
    df['MONTOCONCESION_ABS'] = abs(df['MONTOCONCESION'])
    df['DIFERENCIA_ABSOLUTA_MONTO_RECLAMADO_AUTORIZADO'] = abs(df['DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO'])
    



diferencia_montos(df)  

In [ ]:
df.head()

In [77]:
def calcular_fechas_1(df, salas_list):
    
    # Calcular Dias de habitación
    dias_habitacion_mask = df['PRESTACION'].isin(salas_list)
    df.loc[dias_habitacion_mask, 'DIAS_HABITACION'] = df['CANTIDAD']
    
    # Calcular la fecha de fin de servicio
    def calculate_end_date(ser):
        if ser['DIAS_HABITACION']== np.nan:
            return 
        
        start_date = ser['FECHASERVICIO']
        dias_habitacion = ser['CANTIDAD']
        end_date = start_date + datetime.timedelta(days=dias_habitacion)
        return end_date
        
    df['FECHASERVICIO_FIN'] = df.apply(lambda x:calculate_end_date(x), axis=1)
    

calcular_fechas_1(df, salas_list)

In [ ]:
print(df.shape)


In [ ]:
df.head(1)

In [78]:
# Alertas para Reglas de Negocio, relacionada a montos

especialidades_claves = [
    'CENTROS MEDICOS',
    'ATENCION FARMACEUTICA', 
    'CTROS. DIAGNOSTICOS',
    'MEDICINA FISICA Y REHABILITACION', 
    'LABORATORIO CLINICO DE HEMATOLOGIA Y BAN',
]

def tipo_especialidad(df, especialidades_claves):
    df['TIPO_ESPECIALIDAD'] = df['ESPECIALIDAD']
    df.loc[~df['TIPO_ESPECIALIDAD'].isin(especialidades_claves), 'TIPO_ESPECIALIDAD'] = 'OTROS'
    

tipo_especialidad(df, especialidades_claves)

In [ ]:
df.shape

In [79]:
def alertas_montos_1(df):
    # Alertas para Reglas de Negocio, relacionada a montos
    df['ERROR_MONTO_RECLAMADO'] = np.where(df['DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO_USUARIO'] != 0, 1, 0)
    df['TIENE_MONTO_EXCEPCION'] = np.where(abs(df['MONTOEXCEPCION']) > 0, 1, 0)
    df['ERROR_MONTO_AUTORIZADO2'] = np.where(abs(df['MONTOAUTORIZADO'] - df['MONTOASEGURADORA'] - df['MONTOCONCESION'] - df['MONTOEXCEPCION']) > 0.1, 1, 0)
    df['ERROR_CONSECION'] = np.where(df['MONTOCONCESION_ABS'] > 0.2 * df['MONTOAUTORIZADO'], 1, 0)
    df['ERROR_MONTO_AUTORIZADO1'] = np.where(df['MONTOAUTORIZADO'] > df['MONTORECLAMADO'], 1, 0)
    
alertas_montos_1(df)

In [ ]:
print(df.shape)


In [ ]:
df.head(1)

In [80]:
def alertas_dias_habitacion(df):
    # Alertas para Reglas de Negocio, relacionada a los dias de habitación
    df['LIM_2_DIAS_HABITACION_EXCEDIDO'] = np.where(df['DIAS_HABITACION'] > 2, 1, 0)
    df['LIM_6_DIAS_HABITACION_EXCEDIDO'] = np.where(df['DIAS_HABITACION'] > 6, 1, 0)
    
alertas_dias_habitacion(df)

In [ ]:
print(df.shape)


In [ ]:
df.head(1)

In [81]:
def indicadores_de_atencion(df):
    df['ATENCION_AMBULATORIA'] = np.where(df['TIPOATENCION'] == 'AMBULATORIA', 1, 0)
    df['ATENCION_HOSPITALARIA'] = np.where(df['TIPOATENCION'] == 'HOSPITALARIA', 1, 0)
    
indicadores_de_atencion(df)
print(df.shape)


(17614, 41)


In [ ]:
df.head(1)

In [82]:
def indicadores_de_prestador(df):
    # Indicadores de prestador
    df['CENTROS_MEDICOS'] = np.where(df['TIPO_ESPECIALIDAD'] == 'CENTROS MEDICOS', 1, 0)
    df['CENTRO_DIAGNOSTICO'] = np.where(df['TIPO_ESPECIALIDAD'] == 'CTROS. DIAGNOSTICOS', 1, 0)
    df['LABORATORIO_CLINICO'] = np.where(df['TIPO_ESPECIALIDAD'] == 'LABORATORIO CLINICO DE HEMATOLOGIA Y BAN', 1, 0)
    df['ATENCION_FARMACEUTICA'] = np.where(df['TIPO_ESPECIALIDAD'] == 'ATENCION FARMACEUTICA', 1, 0)
    df['OTRA_ESPECIALIDAD'] = np.where(df['TIPO_ESPECIALIDAD'] == 'OTROS', 1, 0)
    df['MEDICO_ESPECIALISTA'] = np.where(df['TIPOPRESTADOR'] == 'MEDICOS Y/O ESPECIALISTAS', 1, 0)
    df['ES_CONSULTA'] = np.where(df['PRESTACION'].str.contains('CONSULTA'), 1, 0)
    
indicadores_de_prestador(df)
print(df.shape)


(17614, 48)


In [ ]:
df.head(1)

In [83]:
def indicadores_de_procedimiento(df):
    # Indicadores de procedimiento
    df['OTROS_SERVICIOS'] = np.where(df['PRESTACION'] == 'OTROS SERVICIOS NO CUBIERTOS', 1, 0)
    df['MEDICINAS_INTERNAMIENTO'] = np.where(df['PRESTACION'] == 'Medicinas Durante Internamiento todo tipo', 1, 0)
    df['MATERIALES_DESECHABLES'] = np.where(df['PRESTACION'] == 'MATERIALES DESECHABLES', 1, 0)
    

def metricas(df):
    df['MONTO_USUARIO_PER_CAPITA'] = df['MONTOUSUARIO'] / df['CANTIDAD']
    df['DIFERENCIA_RECLAMADO_AUTORIZADO'] = df['MONTORECLAMADO'] - df['MONTOAUTORIZADO']
    

indicadores_de_procedimiento(df)
metricas(df)
print(df.shape)

(17614, 53)


In [ ]:
df.head(1)

In [ ]:
df.loc[df['AUTORIZACION']==42316928].head(1)

In [84]:
# Calcular prestaciones que exceden el maximo esperado.

def calcular_prestaciones_excedidas(consolidated_df, valores_referencia_cups_df):
    # Combinar los DataFrames en función de la columna 'PRESTACION' en consolidated_df
    # y la columna 'cupsNombre' en valores_referencia_cups_df
    merged_df = consolidated_df.merge(
        valores_referencia_cups_df[['cupsNombre', 'cupsMaxVecesDia']],
        left_on='PRESTACION',
        right_on='cupsNombre',
        how='left'
    )

    # Crear una nueva columna 'LIM_CANTIDAD_PRESTACION_EXCEDIDO' que es 1 si 'CANTIDAD'
    # excede 'cupsMaxVecesDia' y 0 en caso contrario
    merged_df['LIM_CANTIDAD_PRESTACION_EXCEDIDO'] = (merged_df['CANTIDAD'] > merged_df['cupsMaxVecesDia']).astype(int)
    
    # Eliminar las columnas innecesarias del resultado
    result_df = merged_df.drop(columns=['cupsMaxVecesDia', 'cupsNombre'])
    
    # Actualizar el DataFrame original con el resultado
    consolidated_df[result_df.columns] = result_df


calcular_prestaciones_excedidas(df, valores_referencia_cups_df)
print(df.shape)

(17614, 54)


In [ ]:
df.loc[df['AUTORIZACION']==42316928].head(1)

In [ ]:
df_test = df.copy()

In [ ]:
print(df_test.shape)
print(df.shape)

In [85]:
# Visita a diferente especialista, misma especialidad, mayor a 2 por mes 

def misma_especialidad_diff_especialista(consolidated_df, lim_especialista=2):
    # Filtrar por pacientes que hayan visitado a diferentes especialistas en la misma especialidad más de dos veces al mes
    cant_especialistas_diferentes = consolidated_df[
        (consolidated_df['MEDICO_ESPECIALISTA'] == 1) & (consolidated_df['ES_CONSULTA'] == 1)
    ].groupby(
        ['CLIENTEAFILIADOBENEFICIARIO', 'ESPECIALIDAD', 'FECHASERVICIO_MONTH_NUMBER', 'FECHASERVICIO_MONTH']
    ).agg(
        CANT_ESPECIALISTAS_DIFERENTES=pd.NamedAgg(column='PRESTADOR', aggfunc='nunique'),
        AUTORIZACIONES=pd.NamedAgg(column='AUTORIZACION', aggfunc='unique'),
    ).sort_values(by='CANT_ESPECIALISTAS_DIFERENTES', ascending=False)
    
    # Filtrar por pacientes que hayan visitado a más de dos especialistas diferentes en un mes
    cant_especialistas_diferentes = cant_especialistas_diferentes[cant_especialistas_diferentes['CANT_ESPECIALISTAS_DIFERENTES'] > lim_especialista]
    cant_especialistas_diferentes.reset_index(inplace=True)
    
    # Explode para tener una autorización por fila
    cant_especialistas_diferentes = cant_especialistas_diferentes.explode('AUTORIZACIONES')
    autorizaciones_especialistas_diferentes = cant_especialistas_diferentes['AUTORIZACIONES'].unique()
    
    # Crear nueva columna en consolidated_df
    consolidated_df['MISMA_ESPECIALIDAD_DIFF_ESPECIALISTA'] = (
        (consolidated_df['MEDICO_ESPECIALISTA'] == 1) & 
        (consolidated_df['ES_CONSULTA'] == 1) &
        (consolidated_df['AUTORIZACION'].isin(autorizaciones_especialistas_diferentes) )
    )*1
    
misma_especialidad_diff_especialista(df)
df.shape

(17614, 55)

In [ ]:
df_test.loc[df_test['AUTORIZACION']==42316928].head(1)

In [86]:
# Visita al mismo especialista mayor a 2 por mes

def visita_mismo_especialista(consolidated_df, lim_consultas_mes=2):
    
    # Agrupar por 'CLIENTEAFILIADOBENEFICIARIO', 'PRESTADOR', 'FECHASERVICIO_MONTH_NUMBER' y 'FECHASERVICIO_MONTH'
    cant_visitas_especialista = consolidated_df[
        (consolidated_df['MEDICO_ESPECIALISTA'] == 1) &
        (consolidated_df['ES_CONSULTA'] == 1)
    ].groupby(
        ['CLIENTEAFILIADOBENEFICIARIO', 'PRESTADOR', pd.Grouper(key='FECHASERVICIO', freq='M')]
    ).agg(
        NUM_VISITAS=pd.NamedAgg(column='FECHASERVICIO', aggfunc='nunique'),
        AUTORIZACIONES=pd.NamedAgg(column='AUTORIZACION', aggfunc='unique'),
    ).reset_index()

    # Filtrar por el límite máximo de visitas permitido por mes
    cant_visitas_especialista = cant_visitas_especialista[cant_visitas_especialista['NUM_VISITAS'] > lim_consultas_mes]

    # Expandir la columna 'AUTORIZACIONES'
    cant_visitas_especialista = cant_visitas_especialista.explode('AUTORIZACIONES')

    # Obtener las autorizaciones que exceden el límite de visitas
    autorizaciones_visitas_especialista = cant_visitas_especialista['AUTORIZACIONES'].unique()

    # Agregar el indicador 'LIM_CONSULTAS_EXCEDIDO_MES'
    consolidated_df['LIM_CONSULTAS_EXCEDIDO_MES'] = (
        (consolidated_df['MEDICO_ESPECIALISTA'] == 1) &
        (consolidated_df['ES_CONSULTA'] == 1) &
        (consolidated_df['AUTORIZACION'].isin(autorizaciones_visitas_especialista))
    )*1


visita_mismo_especialista(df)
print(df.shape)

(17614, 56)


In [ ]:
df_test.loc[df_test['AUTORIZACION']==42316928].head(1)

In [ ]:
df.loc[df['AUTORIZACION']==43866305]

In [ ]:
df.shape

In [87]:
## Límite Emergencias excedido en el mes

def limite_emergencias_excedido(df):
    # Crear la columna "ES_EMERGENCIA" usando np.where
    df['ES_EMERGENCIA'] = np.where(df['SERVICIOSALUD'].str.contains('EMERGENCIA'), 1, 0)
    
    # Contar la cantidad de autorizaciones de emergencias por mes y cliente
    freq_emergencias_mes = df.loc[df['ES_EMERGENCIA'] == 1].groupby([
        'CLIENTEAFILIADOBENEFICIARIO', 'FECHASERVICIO_MONTH_NUMBER', 'FECHASERVICIO_MONTH'
    ]).agg({
        'AUTORIZACION': ['nunique', 'unique']
    })
    
    # Desagregar la columna "AUTORIZACIONES" y filtrar las autorizaciones que exceden el límite de emergencias
    freq_emergencias_mes = freq_emergencias_mes.explode(('AUTORIZACION', 'unique'))
    freq_emergencias_mes = freq_emergencias_mes.loc[
        freq_emergencias_mes[('AUTORIZACION', 'nunique')] > 2
    ]
    
    autorizaciones_emergencias_excedida = freq_emergencias_mes[('AUTORIZACION', 'unique')].unique()
    
    # Crear la columna "LIM_EMERGENCIAS_EXCEDIDO_MES" usando np.where
    df['LIM_EMERGENCIAS_EXCEDIDO_MES'] = np.where(
        (df['ES_EMERGENCIA'] == 1) & (df['AUTORIZACION'].isin(autorizaciones_emergencias_excedida)), 1, 0
    )
    


limite_emergencias_excedido(df)
print(df.shape)

(17614, 58)


In [ ]:
df_test.loc[df_test['AUTORIZACION']==42316928].head(1)

In [88]:
# Consultas que no generan medicamento pt1

def df_autorizaciones_cond_consulta(consolidated_df):
    # Filtrar las filas con MEDICO_ESPECIALISTA == 1 y ES_CONSULTA == 1
    consultas_df = consolidated_df[(consolidated_df['MEDICO_ESPECIALISTA'] == 1) & (consolidated_df['ES_CONSULTA'] == 1)]
    
    # Obtener los valores únicos de la columna 'AUTORIZACION' y convertirlos en un conjunto (set)
    autorizaciones_cond_consulta = set(consultas_df['AUTORIZACION'].unique())
    
    return autorizaciones_cond_consulta

# Llamar a la función y guardar el resultado en la variable autorizaciones_cond_consulta
autorizaciones_cond_consulta = df_autorizaciones_cond_consulta(df)



In [ ]:
print(len(autorizaciones_cond_consulta))

In [89]:
# Consultas que no generan medicamento pt2

def generar_nuevas_consultas(consolidated_df, autorizaciones):
    # Filtrar el DataFrame utilizando las autorizaciones del conjunto (set) proporcionado
    consultas_atipicas_df = consolidated_df[consolidated_df['AUTORIZACION'].isin(autorizaciones)]
    consultas_sample = consultas_atipicas_df[['CLIENTEAFILIADOBENEFICIARIO', 'FECHASERVICIO']].dropna()

    # Ajustar la fecha en 16 días
    consultas_sample['FECHASERVICIO'] = consultas_sample['FECHASERVICIO'] - pd.offsets.Day(16)

    records = []

    # Iterar sobre las filas de consultas_sample y generar un DataFrame con nuevas fechas
    for _, row in consultas_sample.iterrows():
        tmp_usuario = row['CLIENTEAFILIADOBENEFICIARIO']
        tmp_fecha_inicio = row['FECHASERVICIO']

        tmp_df = pd.DataFrame()
        tmp_df['FECHASERVICIO'] = pd.date_range(tmp_fecha_inicio, periods=32, freq='D')
        tmp_df['CLIENTEAFILIADOBENEFICIARIO'] = tmp_usuario

        records += tmp_df.to_records(index=False).tolist()

    # Crear un nuevo DataFrame con las nuevas fechas de consulta
    new_consultas_dates_df = pd.DataFrame(
        records,
        columns=['FECHASERVICIO', 'CLIENTEAFILIADOBENEFICIARIO'],
    )

    # Convertir la columna 'FECHASERVICIO' al tipo de dato datetime
    new_consultas_dates_df['FECHASERVICIO'] = pd.to_datetime(
        new_consultas_dates_df['FECHASERVICIO'],
        errors='coerce'
    )

    return new_consultas_dates_df

# Llamar a la función y guardar el resultado en la variable new_consultas_dates_df
new_consultas_dates_df = generar_nuevas_consultas(df, autorizaciones_cond_consulta)


In [90]:
# Consultas que no generan medicamento pt3

def agregar_columna_consultas_sin_lab_med(consolidated_df, new_consultas_dates_df):
    # Combinar consolidated_df y new_consultas_dates_df usando CLIENTEAFILIADOBENEFICIARIO y FECHASERVICIO
    consultas_tracking_df = consolidated_df.merge(
        new_consultas_dates_df,
        on=['CLIENTEAFILIADOBENEFICIARIO', 'FECHASERVICIO']
    )

    # Agrupar por cliente y contar el número de prestaciones
    sample = consultas_tracking_df.groupby(['CLIENTEAFILIADOBENEFICIARIO']).agg(
        NUM_PRESTACIONES=pd.NamedAgg(column='PRESTACION', aggfunc='count'),
    )

    # Filtrar los usuarios con una única prestación
    usuarios = sample[sample['NUM_PRESTACIONES'] == 1].index.tolist()

    # Filtrar el DataFrame por los usuarios y excluir las filas con la especialidad 'PSICOLOGIA'
    sample_consultas_tracking_df = consultas_tracking_df[
        consultas_tracking_df['CLIENTEAFILIADOBENEFICIARIO'].isin(usuarios)
    ]
    sample_consultas_tracking_df = sample_consultas_tracking_df[
        sample_consultas_tracking_df['ESPECIALIDAD'] != 'PSICOLOGIA'
    ]

    # Obtener las autorizaciones únicas
    autorizaciones_cond_consulta_sin_lab_meds = sample_consultas_tracking_df['AUTORIZACION'].unique()

    # Agregar la columna 'CONSULTAS_SIN_LAB_MED' al DataFrame original
    consolidated_df['CONSULTAS_SIN_LAB_MED'] = (
        consolidated_df['AUTORIZACION'].isin(autorizaciones_cond_consulta_sin_lab_meds)
    ) * 1

# Llamar a la función para aplicar los cambios en el DataFrame original
agregar_columna_consultas_sin_lab_med(df, new_consultas_dates_df)
print(df.shape)

(17614, 59)


In [ ]:
df_test.loc[df_test['AUTORIZACION']==42316928].head(1)

In [91]:
# Distribución de medicamentos > 20% pt1

def calcular_porcentaje_tipo_prestacion(consolidated_df):
    # Asignar el valor de la columna AUTORIZACION a AUTORIZACIONRELACIONADA donde AUTORIZACION está en autorizaciones_padres_list
    autorizaciones_padres_list = consolidated_df['AUTORIZACIONRELACIONADA'].dropna().unique().tolist()
    consolidated_df.loc[consolidated_df['AUTORIZACION'].isin(autorizaciones_padres_list), 'AUTORIZACIONRELACIONADA'] = consolidated_df['AUTORIZACION']

    # Agrupar y calcular las métricas para cada grupo
    eventos_agrupados_df = consolidated_df.groupby(
        ['AUTORIZACIONRELACIONADA', 'TIPOPRESTACION']
    ).agg(
        NUM_AUTORIZACIONES=pd.NamedAgg(column='AUTORIZACION', aggfunc='nunique'),
        NUM_PROCEDIMIENTOS=pd.NamedAgg(column='PRESTACION', aggfunc='count'),
        MONTO_RECLAMADO=pd.NamedAgg(column='MONTORECLAMADO', aggfunc='sum'),
        MONTO_AUTORIZADO=pd.NamedAgg(column='MONTOAUTORIZADO', aggfunc='sum'),
    ).sort_values(by=['MONTO_RECLAMADO'], ascending=False)

    # Crear una tabla dinámica con MONTO_RECLAMADO como valor
    eventos_agrupados_pivot_df = eventos_agrupados_df.pivot_table(
        values=['MONTO_RECLAMADO'],
        columns=['TIPOPRESTACION'],
        index='AUTORIZACIONRELACIONADA'
    ).fillna(0)

    # Calcular el total de MONTO_RECLAMADO para cada AUTORIZACIONRELACIONADA
    eventos_agrupados_pivot_df['TOTAL'] = eventos_agrupados_pivot_df.sum(axis='columns')

    # Calcular el porcentaje de MONTO_RECLAMADO por TIPOPRESTACION
    eventos_agrupados_pivot_percent_df = (eventos_agrupados_pivot_df.T / eventos_agrupados_pivot_df['TOTAL'].T).T
    eventos_agrupados_pivot_percent_df.columns = eventos_agrupados_pivot_percent_df.columns.droplevel(0)
    eventos_agrupados_pivot_percent_df.reset_index(inplace=True)

    return eventos_agrupados_pivot_percent_df

# Llamar a la función y guardar el resultado en la variable eventos_agrupados_pivot_percent_df
eventos_agrupados_pivot_percent_df = calcular_porcentaje_tipo_prestacion(df)



In [ ]:
eventos_agrupados_pivot_percent_df.shape

In [ ]:
eventos_agrupados_pivot_percent_df.head()

In [ ]:
dftest.shape

In [92]:
# Distribución de medicamentos > 20% pt2

def calcular_lim_dist_meds_excedido(df, eventos_agrupados_pivot_percent_df):
    # Preparar las variables para el proceso de "melt"
    value_vars = eventos_agrupados_pivot_percent_df.columns.tolist()
    id_vars = value_vars.pop(0)
    
    # Aplicar el proceso de "melt" en el DataFrame
    eventos_melt = pd.melt(
        eventos_agrupados_pivot_percent_df,
        id_vars=id_vars,
        value_vars=value_vars,
    )

    # Renombrar y filtrar el DataFrame
    eventos_melt = eventos_melt.rename(columns={'value': '% Total Monto del Evento'})
    eventos_melt = eventos_melt[eventos_melt['TIPOPRESTACION'] != '']
    eventos_melt = eventos_melt[eventos_melt['% Total Monto del Evento'] > 0]
    
    # Aplicar la máscara y obtener los outliers
    eventos_melt_mask = (eventos_melt["% Total Monto del Evento"] > 0.2) & (eventos_melt["TIPOPRESTACION"].str.contains('MEDICIMENTOS'))
    eventos_melt_outliers = eventos_melt[eventos_melt_mask]

    # Obtener las autorizaciones únicas
    autorizaciones_padres_dist_meds = eventos_melt_outliers.AUTORIZACIONRELACIONADA.unique()

    # Calcular la columna LIM_DIST_MEDS_EXCEDIDO y agregarla al DataFrame
    df['LIM_DIST_MEDS_EXCEDIDO'] = (df['AUTORIZACIONRELACIONADA'].isin(autorizaciones_padres_dist_meds)) * 1
    
    return df

# Llamar a la función y actualizar el DataFrame consolidated_df
calcular_lim_dist_meds_excedido(df, eventos_agrupados_pivot_percent_df)
print(df.shape)

(17614, 60)


In [ ]:
df_test['LIM_DIST_MEDS_EXCEDIDO'].value_counts()

In [ ]:
dftest.shape

In [93]:
# Montos Atípicos


def calcular_atipicos(df: pd.DataFrame, columnas: list) -> pd.DataFrame:
    for columna in columnas:
        x_df = df[columna]
        
        # Calcular el primer cuartil, tercer cuartil e IQR (rango intercuartil)
        Q1 = x_df.quantile(q=.25)
        Q3 = x_df.quantile(q=.75)
        IQR = Q3 - Q1

        # Calcular la columna ATIPICO para cada columna en la lista y agregarla al DataFrame
        df['ATIPICO_' + columna] = ((x_df < (Q1 - 2 * IQR)) | (x_df > (Q3 + 2 * IQR))) * 1



# Columnas para las cuales calcular valores atípicos
montos = [
    'MONTOAJUSTADO', 
    'MONTOUSUARIO',
    'MONTOASEGURADORA',
    'MONTOEXCEPCION',
    'MONTOCONCESION',
    'MONTOAUTORIZADO',
    'MONTOAJUSTADO_ABS',
    'MONTOCONCESION_ABS',
    'MONTO_USUARIO_PER_CAPITA'
]

# Llamar a la función y actualizar el DataFrame consolidated_df
calcular_atipicos(df, montos)


In [94]:
df.shape

(17614, 69)

In [95]:
jorge = pd.read_parquet('../Datos/consolidated_df_jorge.parquet')

In [96]:
jorge.shape

(17614, 69)

In [31]:
if df.equals(consolidated_df):
    print("Los DF son iguales")
else:
    print("Los DF SON DIFERENTES!")

Los DF SON DIFERENTES!


In [97]:
jorge.head(1)

,PRESTACION,CLIENTEAFILIADOBENEFICIARIO,AUTORIZACION,FECHASERVICIO,TIPOATENCION,PRESTADOR,TIPOPRESTADOR,ESPECIALIDAD,TIPOPRESTACION,SERVICIOSALUD,...,LIM_DIST_MEDS_EXCEDIDO,ATIPICO_MONTOAJUSTADO,ATIPICO_MONTOUSUARIO,ATIPICO_MONTOASEGURADORA,ATIPICO_MONTOEXCEPCION,ATIPICO_MONTOCONCESION,ATIPICO_MONTOAUTORIZADO,ATIPICO_MONTOAJUSTADO_ABS,ATIPICO_MONTOCONCESION_ABS,ATIPICO_MONTO_USUARIO_PER_CAPITA
0,ACIDO URICO< O:P>,N-644925,47527093,2022-07-26,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0


In [98]:
df.head(1)

,CLIENTEAFILIADOBENEFICIARIO,AUTORIZACION,FECHASERVICIO,TIPOATENCION,PRESTADOR,TIPOPRESTADOR,ESPECIALIDAD,PRESTACION,TIPOPRESTACION,SERVICIOSALUD,...,LIM_DIST_MEDS_EXCEDIDO,ATIPICO_MONTOAJUSTADO,ATIPICO_MONTOUSUARIO,ATIPICO_MONTOASEGURADORA,ATIPICO_MONTOEXCEPCION,ATIPICO_MONTOCONCESION,ATIPICO_MONTOAUTORIZADO,ATIPICO_MONTOAJUSTADO_ABS,ATIPICO_MONTOCONCESION_ABS,ATIPICO_MONTO_USUARIO_PER_CAPITA
0,N-1366372,46857903,2022-05-18,AMBULATORIA,CLINICA DR. PEROZO SRL,"CLINICAS, HOSPITALES Y CENTROS MEDICOS",CENTROS MEDICOS,MATERIALES DESECHABLES,SERVICIOS ESPECIALES DE HOSPITALIZACION,EMERGENCIA TRIAGE III,...,0,0,0,0,0,0,0,0,0,0


In [99]:
# Extraer los nombres de columnas de ambos DataFrames
df_columns = df.columns.tolist()
jorge_columns = jorge.columns.tolist()

# Ordenar alfabéticamente los nombres de columnas
df_columns.sort()
jorge_columns.sort()

# Comparar si los nombres de columnas son iguales
if df_columns == jorge_columns:
    print("Los nombres de columnas son iguales.")
else:
    print("Los nombres de columnas son diferentes.")


Los nombres de columnas son iguales.


In [110]:
df_columns

['ATENCION_AMBULATORIA',
 'ATENCION_FARMACEUTICA',
 'ATENCION_HOSPITALARIA',
 'ATIPICO_MONTOAJUSTADO',
 'ATIPICO_MONTOAJUSTADO_ABS',
 'ATIPICO_MONTOASEGURADORA',
 'ATIPICO_MONTOAUTORIZADO',
 'ATIPICO_MONTOCONCESION',
 'ATIPICO_MONTOCONCESION_ABS',
 'ATIPICO_MONTOEXCEPCION',
 'ATIPICO_MONTOUSUARIO',
 'ATIPICO_MONTO_USUARIO_PER_CAPITA',
 'AUTORIZACION',
 'AUTORIZACIONRELACIONADA',
 'CANTIDAD',
 'CENTROS_MEDICOS',
 'CENTRO_DIAGNOSTICO',
 'CLIENTEAFILIADOBENEFICIARIO',
 'CONSULTAS_SIN_LAB_MED',
 'DIAS_HABITACION',
 'DIFERENCIA_ABSOLUTA_MONTO_RECLAMADO_AUTORIZADO',
 'DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO',
 'DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO_USUARIO',
 'DIFERENCIA_RECLAMADO_AUTORIZADO',
 'ERROR_CONSECION',
 'ERROR_MONTO_AUTORIZADO1',
 'ERROR_MONTO_AUTORIZADO2',
 'ERROR_MONTO_RECLAMADO',
 'ESPECIALIDAD',
 'ES_CONSULTA',
 'ES_EMERGENCIA',
 'FECHASERVICIO',
 'FECHASERVICIO_FIN',
 'FECHASERVICIO_MONTH',
 'FECHASERVICIO_MONTH_NUMBER',
 'LABORATORIO_CLINICO',
 'LIM_2_DIAS_HABITACION_EXCEDIDO'

In [109]:
jorge

,PRESTACION,CLIENTEAFILIADOBENEFICIARIO,AUTORIZACION,FECHASERVICIO,TIPOATENCION,PRESTADOR,TIPOPRESTADOR,ESPECIALIDAD,TIPOPRESTACION,SERVICIOSALUD,...,LIM_DIST_MEDS_EXCEDIDO,ATIPICO_MONTOAJUSTADO,ATIPICO_MONTOUSUARIO,ATIPICO_MONTOASEGURADORA,ATIPICO_MONTOEXCEPCION,ATIPICO_MONTOCONCESION,ATIPICO_MONTOAUTORIZADO,ATIPICO_MONTOAJUSTADO_ABS,ATIPICO_MONTOCONCESION_ABS,ATIPICO_MONTO_USUARIO_PER_CAPITA
0,ACIDO URICO< O:P>,N-644925,47527093,2022-07-26,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0
1,ACIDO URICO< O:P>,N-796869,47570405,2022-08-01,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0
2,ACIDO URICO< O:P>,N-239664,46076598,2022-02-24,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0
3,ACIDO URICO< O:P>,N-525689,42899323,2021-03-06,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0
4,ACIDO URICO< O:P>,N-644925,45793382,2022-01-26,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17609,VITAMINA B 12,N-241627,43868718,2021-06-23,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0
17610,VITAMINA B 12,N-525689,42899323,2021-03-06,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0
17611,VITAMINA D 25 DIHIDROXI,N-644925,47527093,2022-07-26,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0
17612,VITAMINA D 25 DIHIDROXI,N-241627,43868718,2021-06-23,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0


In [111]:
jorge.to_csv('jorge.csv')

In [100]:
alex = df.copy()

In [101]:
# Extraer los nombres de las columnas de "consolidated_df"
column_names = list(jorge.columns)

# Reordenar las columnas en "df" según los nombres de columnas en "consolidated_df"
alex = alex[column_names]


In [104]:
alex_m = alex.loc[alex['AUTORIZACION']==47527093].reset_index().drop('index', axis=1).sort_values(by='PRESTACION')

In [105]:
jorge_m = jorge.loc[jorge['AUTORIZACION']==47527093].reset_index().drop('index', axis=1).sort_values(by='PRESTACION')

In [106]:
print(alex_m.shape)
print(jorge_m.shape)

(31, 69)
(31, 69)


In [64]:
a.head(1)

,PRESTACION,CLIENTEAFILIADOBENEFICIARIO,AUTORIZACION,FECHASERVICIO,TIPOATENCION,PRESTADOR,TIPOPRESTADOR,ESPECIALIDAD,TIPOPRESTACION,SERVICIOSALUD,...,LIM_DIST_MEDS_EXCEDIDO,ATIPICO_MONTOAJUSTADO,ATIPICO_MONTOUSUARIO,ATIPICO_MONTOASEGURADORA,ATIPICO_MONTOEXCEPCION,ATIPICO_MONTOCONCESION,ATIPICO_MONTOAUTORIZADO,ATIPICO_MONTOAJUSTADO_ABS,ATIPICO_MONTOCONCESION_ABS,ATIPICO_MONTO_USUARIO_PER_CAPITA
20,ACIDO URICO< O:P>,N-644925,47527093,2022-07-26,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0


In [65]:
b.head(1)

,PRESTACION,CLIENTEAFILIADOBENEFICIARIO,AUTORIZACION,FECHASERVICIO,TIPOATENCION,PRESTADOR,TIPOPRESTADOR,ESPECIALIDAD,TIPOPRESTACION,SERVICIOSALUD,...,LIM_DIST_MEDS_EXCEDIDO,ATIPICO_MONTOAJUSTADO,ATIPICO_MONTOUSUARIO,ATIPICO_MONTOASEGURADORA,ATIPICO_MONTOEXCEPCION,ATIPICO_MONTOCONCESION,ATIPICO_MONTOAUTORIZADO,ATIPICO_MONTOAJUSTADO_ABS,ATIPICO_MONTOCONCESION_ABS,ATIPICO_MONTO_USUARIO_PER_CAPITA
0,ACIDO URICO< O:P>,N-644925,47527093,2022-07-26,AMBULATORIA,"LABORATORIO DE REFERENCIA, S. A.",PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0,0,0,0,0,0,0,0,0,0


In [107]:
if alex_m.equals(jorge_m):
    print("Los DF son iguales")
else:
    print("LOS DF SON DIFERENTES")

LOS DF SON DIFERENTES


In [108]:
alex_m.to_csv('alex_m.csv')
jorge_m.to_csv('jorge_m.csv')

# Cálculo de las métricas previas a la predicción

In [ ]:
measures = {
    'CANTIDAD_AUTORIZACIONES': pd.NamedAgg(column='AUTORIZACION', aggfunc='nunique'),
    'LIM_CANTIDAD_PRESTACION_EXCEDIDO': pd.NamedAgg(column='LIM_CANTIDAD_PRESTACION_EXCEDIDO', aggfunc='sum'), #
    'MISMA_ESPECIALIDAD_DIFF_ESPECIALISTA': pd.NamedAgg(column='MISMA_ESPECIALIDAD_DIFF_ESPECIALISTA', aggfunc='sum'), #
    'LIM_CONSULTAS_EXCEDIDO_MES': pd.NamedAgg(column='LIM_CONSULTAS_EXCEDIDO_MES', aggfunc='sum'), #
    'ES_EMERGENCIA': pd.NamedAgg(column='ES_EMERGENCIA', aggfunc='sum'),  
    'LIM_EMERGENCIAS_EXCEDIDO_MES': pd.NamedAgg(column='LIM_EMERGENCIAS_EXCEDIDO_MES', aggfunc='sum'), #     
    'ATIPICO_MONTO_ASEGURADORA': pd.NamedAgg(column='ATIPICO_MONTOASEGURADORA', aggfunc='sum'), # N/A
    'ATIPICO_MONTO_CONCESION': pd.NamedAgg(column='ATIPICO_MONTOCONCESION', aggfunc='sum'), # N/A
    'ATIPICO_MONTO_EXCEPCION': pd.NamedAgg(column='ATIPICO_MONTOEXCEPCION', aggfunc='sum'), # N/A
    'ATIPICO_MONTO_USUARIO': pd.NamedAgg(column='ATIPICO_MONTOUSUARIO', aggfunc='sum'), # N/A
    'ATIPICO_MONTO_AJUSTADO': pd.NamedAgg(column='ATIPICO_MONTOAJUSTADO', aggfunc='sum'), # N/A
    'ATIPICO_MONTO_USUARIO_PER_CAPITA': pd.NamedAgg(column='ATIPICO_MONTO_USUARIO_PER_CAPITA', aggfunc='sum'), #N/A
    'ERRORES_MONTO_RECLAMADO': pd.NamedAgg(column='ERROR_MONTO_RECLAMADO', aggfunc='sum'),
    'ERRORES_MONTO_AUTORIZADO1': pd.NamedAgg(column='ERROR_MONTO_AUTORIZADO1', aggfunc='sum'),
    'ERRORES_MONTO_AUTORIZADO2': pd.NamedAgg(column='ERROR_MONTO_AUTORIZADO2', aggfunc='sum'),
    'CANTIDAD_DIAS_HABITACION': pd.NamedAgg(column='DIAS_HABITACION', aggfunc='sum'),
    'LIM_6_DIAS_HABITACION_EXCEDIDO': pd.NamedAgg(column='LIM_6_DIAS_HABITACION_EXCEDIDO', aggfunc='sum'),
    'PRESTACION_MEDICINAS_INTERNAMIENTO': pd.NamedAgg(column='MEDICINAS_INTERNAMIENTO', aggfunc='sum'),
    'PRESTACION_MATERIALES_DESECHABLES': pd.NamedAgg(column='MATERIALES_DESECHABLES', aggfunc='sum'),
    'PRESTACION_OTROS_SERVICIOS': pd.NamedAgg(column='OTROS_SERVICIOS', aggfunc='sum'),
    'N_PRESTACIONES': pd.NamedAgg(column='PRESTACION', aggfunc='count'),        
    'ERRORES_CONSECION': pd.NamedAgg(column='ERROR_CONSECION', aggfunc='sum'),
    'CONSULTAS_SIN_LAB_MED': pd.NamedAgg(column='CONSULTAS_SIN_LAB_MED', aggfunc='sum'), #
    'LIM_DIST_MEDS_EXCEDIDO': pd.NamedAgg(column='LIM_DIST_MEDS_EXCEDIDO', aggfunc='sum') #
   }

df_medidas = dftest.groupby(
    ['PRESTADOR', 'TipoInstitucionPrestador']
).agg(
    **measures
).sort_values(
    by='CANTIDAD_AUTORIZACIONES', 
    ascending=False
).reset_index()

In [ ]:
df_medidas.shape

In [ ]:
df_medidas.head(3)